# Plotting the RMSE

In [3]:
# Initialize
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

In [4]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.conv1 = nn.Conv2d(3, 40, 5)
        self.pool = nn.MaxPool2d(4, 4)
        self.drop = nn.Dropout(p=0.5, inplace=False)
        self.conv2 = nn.Conv2d(40, 60, 5)
        self.conv3 = nn.Conv2d(60, 240, 3)
        self.resnet = nn.Linear(240, 240)  # NOT ACTUALLY RESNET
        self.fc1 = nn.Linear(240, 120)
        self.fc2 = nn.Linear(120, 60)
        self.fc3 = nn.Linear(60, 1)
    
    def forward(self, x):
        x = self.pool(F.tanh(self.conv1(x)))
        x = self.pool(F.tanh(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        # print(x.shape)
        # x = self.drop(x)

        # Best result so far
        x = F.relu(self.resnet(x))
        x = F.relu(self.resnet(x))



        # 1 Layer Resnet did perform very well
        # x = F.relu(self.resnet(F.relu(self.resnet(x)))) + x
        # x = F.relu(self.resnet(F.relu(self.resnet(x)))) + x
        # x = self.drop(x)
        # x = F.relu(self.resnet(F.relu(self.resnet(x)))) + x
        # x = self.drop(x)
        # x = F.relu(self.resnet(F.relu(self.resnet(x)))) + x
        # x = self.drop(x)
        # x = F.relu(self.resnet(F.relu(self.resnet(x)))) + x


        x = F.relu(self.fc1(x))
        # x = self.drop(x)
        x = F.relu(self.fc2(x))
        # x = self.drop(x)
        x = self.fc3(x)
        # no activation and no softmax at the end
        return x
    


In [ ]:
#for visualization we will use vgg16 pretrained on imagenet data

device = torch.device('cpu')
model = NeuralNet(57600, 1)
model.load_state_dict(torch.load('/home/jiawenb/CS229/cs229-project/CNN_MLP_trained_model.pt', map_location=device))
model.eval()

In [6]:
#plt.plot()
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)

def generate_boxplot(all_outputs, all_labels):
    # Convert numpy arrays to hashable types
    all_labels = [tuple(label) for label in all_labels]

    # Create a dictionary to map labels to integer values
    label_mapping = {label: i for i, label in enumerate(set(all_labels))}

    # Create empty lists to store binned data points
    binned_data = [[] for _ in label_mapping]

    # Iterate through the outputs and labels and bin them based on the labels
    for outputs, labels in zip(all_outputs, all_labels):
        bin_index = label_mapping[labels]

        # Flatten the outputs if it's a multidimensional array
        if outputs.ndim > 1:
            outputs = outputs.flatten()

        binned_data[bin_index].extend(outputs)

    # Generate a box and whisker plot for each set of binned data
    plt.figure()
    plt.boxplot(binned_data, labels=label_mapping.keys())
    plt.xlabel('Labels')
    plt.ylabel('Outputs')
    plt.title('Box and Whisker Plot')
    plt.show()

In [ ]:
loss = 0
with torch.no_grad():
    all_outputs = []
    all_labels = []
    all_error = []
    loss = 0

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.unsqueeze(1).to(device)
        outputs = model(images)

        all_outputs.extend(outputs.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_error.extend(abs(outputs.cpu().numpy()-labels.cpu().numpy())/labels.cpu().numpy())

        # Calculate RMSE
        delloss = criterion(outputs, labels)
        loss += delloss.item() * images.size(0)

    plt.scatter(all_outputs, all_labels, s=3, c="red")
    plt.xlabel('Predicted Velocity (mph)')
    plt.ylabel('Velocity Label (mph)')
    plt.show()

    plt.scatter(all_labels, all_error, s=3, c="red")
    plt.xlabel('Velocity (mph)')
    plt.ylabel('Absolute Error(mph)')
    plt.show()

    loss /= len(test_loader.sampler)
    print(f'MSE on the test data is: {loss}')
    loss = np.sqrt(loss)
    print(f'RMSE on the test data is: {loss}')